In [1]:
%run PilotForestAreas.ipynb

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
import param as pm
import panel as pn
import pandas as pd
import datetime as dt
import numpy as np
pn.extension()

In [3]:
f1.sale_value()

3118225.187825

In [4]:
f1.forested_area_hectares

7478

In [5]:
f1.acres_protected()

18470.66

In [6]:
f1.tons_carbon_per_year

21867.58

In [7]:
class Carbon(pm.Parameterized):
    initial_carbon_price = pm.Number(29)
    carbon_price_appreciation = pm.Number(0.15, bounds=(0,1), step=0.01)
    stewardship_budget_increase = pm.Number(0.075, bounds=(0,1), step=0.01)
    tax_rate = pm.Number(0.2, bounds=(0,1), step=0.01)
    centree_admin_commission = pm.Number(0.15, bounds=(0,1), step=0.01)
    pfa_commission = pm.Number(0.425, bounds=(0,1), step=0.01)
    land_title_holders_commission = pm.Number(0.425, bounds=(0,1), step=0.01)
    below_soil_tonnage_factor = pm.Number(1, bounds=(0,2), step=0.01)
    
    def compute(self, forest_area: Forest, periods = 10, freq = 'Y'):
        
        df = pd.DataFrame(index=pd.date_range(dt.datetime.now(), periods=periods, freq=freq))
        df['Carbon Price'] = [self.initial_carbon_price * (1+self.carbon_price_appreciation)**x for x in range(periods)]

        df['Gross Revenue'] = df['Carbon Price'] * forest_area.tons_carbon_per_year * (1 + self.below_soil_tonnage_factor)

        df['Net Revenue'] = df['Gross Revenue'] * (1 - self.tax_rate)

        df['Stewardship Budget'] = [forest_area.forested_area_hectares * forest_area.stewardship_rate_per_hectare * (1+self.stewardship_budget_increase)**x for x in range(periods)]

        df['Gross Profit'] = df['Net Revenue'] - df['Stewardship Budget']

        df['Centree (Admin/Commission)'] = df['Gross Profit'] * self.centree_admin_commission

        df['PFA Commission'] = df['Gross Profit'] * self.pfa_commission
        df['Land Title Holders Commission'] = df['Gross Profit'] * self.land_title_holders_commission

        df['PFA Holder Return as Percentage of Initial Cost'] = df['PFA Commission'] / forest_area.sale_value()
        
        df.index.name = "Date"

        return df
    
    def view_model_table(self, forest):
        return self.compute(forest).reset_index().hvplot.table()
    
    def view_model_normalized_chart(self, forest):
        carbon = self.compute(forest)
        carbon_normal = carbon / carbon.iloc[0] 
        carbon_normal = carbon_normal + np.random.randn(*carbon_normal.shape) / 10
        return pn.panel(carbon_normal.hvplot.line(title="Normalized Growth Projections of Carbon Revenue", height=340, width=1000))

In [8]:
carbon_model = Carbon(below_soil_tonnage_factor=0, carbon_price_appreciation=0.15)

In [9]:
carbon_model.view_model_table(f1)

:Table   [Date,Carbon Price,Gross Revenue,Net Revenue,Stewardship Budget,Gross Profit,Centree (Admin/Commission),PFA Commission,Land Title Holders Commission,PFA Holder Return as Percentage of Initial Cost]

In [10]:
carbon_model.view_model_table(f2)

:Table   [Date,Carbon Price,Gross Revenue,Net Revenue,Stewardship Budget,Gross Profit,Centree (Admin/Commission),PFA Commission,Land Title Holders Commission,PFA Holder Return as Percentage of Initial Cost]

In [11]:
carbon_model.view_model_normalized_chart(f1)

HoloViews(NdOverlay, height=340, sizing_mode='fixed', width=1000)

In [12]:
carbon_model.view_model_normalized_chart(f2)

HoloViews(NdOverlay, height=340, sizing_mode='fixed', width=1000)